In [17]:
import os
from src import utils 
import pandas as pd
import numpy as np

def translate_yaml(yaml_path):
    yaml_data = utils.load_yaml(yaml_path)

    dataset_config = yaml_data.pop('dataset_config')
    model_config = yaml_data.pop('model_config')
    optim_config = yaml_data.pop('optim_config')
    other_config = yaml_data.pop('other_config')

    dataset_config.pop('data_paths')

    yaml_data['data_augmentation'] = ''.join([e[0] for e in yaml_data['data_augmentation']])
    yaml_data['input_data_shape'] = '_'.join([str(e) for e in yaml_data['input_data_shape']])
    yaml_data['output_data_shape'] = '_'.join([str(e) for e in yaml_data['output_data_shape']])
    yaml_data['test_metric_indexes'] = '_'.join([str(e) for e in yaml_data['test_metric_indexes']])

    yaml_data.update(dataset_config)
    yaml_data.update(model_config)
    # yaml_data.update(optim_config)
    yaml_data.update(other_config)

    return yaml_data

def load_test_results(test_metrics_path):
    metric_dict= {}
    for metric in os.listdir(test_metrics_path):
        metric_name = metric[:-4]
        metric_data = np.load(os.path.join(test_metrics_path, metric))
        metric_dict[f'{metric_name}_mean'] = np.mean(metric_data) if len(metric_data) > 0 else 0
        metric_dict[f'{metric_name}_std'] = np.std(metric_data) if len(metric_data) > 0 else 0
    return metric_dict
        

In [18]:
results_path = os.path.abspath('results')

if not os.path.exists(results_path):
    raise ValueError('There is no results folder.')

dataset_list = os.listdir(results_path)
dataset_path_list = [os.path.join(results_path, e) for e in os.listdir(results_path)]

all_results = {}

for dataset, dataset_path in zip(dataset_list, dataset_path_list):
    model_list = os.listdir(dataset_path)
    model_path_list = [os.path.join(dataset_path, e) for e in os.listdir(dataset_path)]
    for model, model_path in zip(model_list, model_path_list):
        additfolder_list = os.listdir(model_path)
        additfolder_path_list = [os.path.join(model_path, e) for e in os.listdir(model_path)]
        cont = 0
        for additfolder, additfolder_path in zip(additfolder_list, additfolder_path_list):
            config_list = os.listdir(additfolder_path)
            config_path_list = [os.path.join(additfolder_path, e) for e in os.listdir(additfolder_path)]
            for config, config_path in zip(config_list, config_path_list):
                yaml_path = os.path.join(config_path, 'train_configuration.yaml')
                test_metrics_path = os.path.join(config_path, 'test_metrics')
                predicted_images_path = os.path.join(config_path, 'predicted_images')
                
                if os.path.exists(yaml_path) and os.path.exists(test_metrics_path):
                    yaml_data = translate_yaml(yaml_path)
                    yaml_data.update(load_test_results(test_metrics_path))
                    all_results[f'{model}_{dataset}_{cont}'] = yaml_data

                    cont += 1


In [20]:
df = pd.DataFrame(all_results).T
nunique = df.nunique()
cols_to_drop = nunique[nunique == 1].index
df.drop(cols_to_drop, axis=1)

,dataset_name,input_data_shape,model,output_data_shape,crappifier,scale,percp_coef,alex_mean,mse_mean,psnr_mean,ssim_mean,vgg_mean
rcan_EM_0,EM,90_64_64_1,rcan,90_256_256_1,em_poisson_crappify,4,1000,0.93002,0.048805,13.11533,0.397067,0.755767
unet_EM_0,EM,90_64_64_1,unet,90_128_128_1,em_poisson_crappify,2,10,0.71156,0.046754,13.301803,0.392829,0.676219
unet_ER_0,ER,288_64_64_1,unet,288_128_128_1,downsampleonly,None,10,0.431936,0.015644,18.05641,0.495008,0.399107
unet_F-actin_0,F-actin,372_64_64_1,unet,372_128_128_1,downsampleonly,None,10,0.699063,0.010929,19.614086,0.545092,0.555629
unet_MitoTracker_small_0,MitoTracker_small,1440_64_64_1,unet,1440_128_128_1,fluo_poisson_crappify,2,10,0.520226,0.028726,15.417203,0.468575,0.546064
